## Import Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.covariance import OAS
from scipy.signal import hilbert
# from scipy.fftpack import hilbert as fhil

## Dataset

In [2]:
abide_dict = np.load('../../dataset/abide.npy', allow_pickle=True).item()
#check the keys in the dictionary
print('Keys in the dictionary:',abide_dict.keys())
print('Timeseries:',abide_dict['timeseires'].shape)
print('Labels:',abide_dict['label'].shape)
print('Correlation:',abide_dict['corr'].shape)
print('PCorr:',abide_dict['pcorr'].shape)
print('Site:',abide_dict['site'].shape)

Keys in the dictionary: dict_keys(['timeseires', 'label', 'corr', 'pcorr', 'site'])
Timeseries: (1009, 200, 100)
Labels: (1009,)
Correlation: (1009, 200, 200)
PCorr: (1009, 200, 200)
Site: (1009,)


## Model

### Covariance Matrix Calculation

In [3]:
def normalized_covariance_matrix(data):
    """
    Calculate the normalized covariance matrix for time series data.
    
    Args:
        data (ndarray): A 2D numpy array containing the time series data.
        
    Returns:
        ndarray: The normalized covariance matrix.
    """
    # Calculate the covariance matrix
    data = hilbert(data) #apply hilbert transform
    cov_matrix = np.cov(data.T, rowvar=False)
    
    # Calculate the diagonal matrix of standard deviations
    std_devs = np.sqrt(np.diag(cov_matrix))
    std_dev_matrix = np.diag(std_devs)
    
    # Calculate the inverse of the standard deviation matrix
    inv_std_dev_matrix = np.linalg.pinv(std_dev_matrix)
    
    # Calculate the normalized covariance matrix
    normalized_cov_matrix = np.dot(inv_std_dev_matrix, np.dot(cov_matrix, inv_std_dev_matrix))
    
    return normalized_cov_matrix


### Classification

#### Tangent Space based SVM

In [4]:
fc_ts = abide_dict['timeseires']
fc_cov = []
for i in range(fc_ts.shape[0]):
    fc_cov.append(normalized_covariance_matrix(fc_ts[i]))
fc_cov = np.array(fc_cov)
print(fc_cov.shape)

(1009, 200, 200)


In [5]:
from pyriemann.estimation import Covariances
from pyriemann.tangentspace import TangentSpace

from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix


# load your data
X = np.imag(fc_cov) # EEG data, in format n_epochs x n_channels x n_times
y = abide_dict['label'] # labels

#train test split

# build your pipeline
covest = Covariances('oas')
# reg_cov = covest.fit_transform(X)
ts = TangentSpace()
# ts_reg_cov = ts.fit_transform(reg_cov)
svc = SVC(kernel='linear')
clf = make_pipeline(covest, ts, svc)
acc = cross_val_score(clf, X, y)
acc.mean()

0.7036746958277919